In [1]:
#general imports
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import os
import os.path as path
from os import listdir 
from os.path import isfile, join
import copy
import pickle
from tqdm import tqdm
import argparse
from collections import namedtuple

In [2]:
# imports from torch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchsummary import summary

import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary

from torchvision.models import vgg16

In [3]:
#Def your own path
ROOT = ""

os.chdir(ROOT)
import sys
sys.path.insert(0, ROOT)

# and here
results_folder = '.'

In [4]:
#Select classes
category_tags = ['mix']
n_objects = len(category_tags) - 1
print('N.of classes : {}'.format(n_objects))

# random generator init
torch.backends.cudnn.deterministic = True
torch.manual_seed(999)

# parameters
arch = 'vgg16'
nsamples = 500
bs = 16

# functions to select checkpoint layers and to determine their depths
# (this works for AlexNet and VGG-like architectures)
def getDepths(model):    
    count = 0    
    modules = []
    names = []
    depths = []    
    modules.append('input')
    names.append('input')
    depths.append(0)    
    
    for i,module in enumerate(model.features):       
        name = module.__class__.__name__
        if 'Conv2d' in name or 'Linear' in name:
            count += 1
        if 'MaxPool2d' in name:
            modules.append(module)
            depths.append(count)
            names.append('MaxPool2d')            
    for i,module in enumerate(model.classifier):
        name = module.__class__.__name__
        if 'Linear' in name:
            modules.append(module)    
            count += 1
            depths.append(count + 1)
            names.append('Linear')                       
    depths = np.array(depths)   
    return modules, names, depths

def getLayerDepth(layer):
    count = 0
    for m in layer:
        for c in m.children():
            name = c.__class__.__name__
            if 'Conv' in name:
                count += 1
    return count

# functions to select checkpoint layers and to determine their depths
# (this works for ResNets architectures)
def getResNetsDepths(model):    
    modules = []
    names = []
    depths = []  
    
    # input
    count = 0
    modules.append('input')
    names.append('input')
    depths.append(count)           
    # maxpooling
    count += 1
    modules.append(model.maxpool)
    names.append('maxpool')
    depths.append(count)     
    # 1 
    count += getLayerDepth(model.layer1)
    modules.append(model.layer1)
    names.append('layer1')
    depths.append(count)         
    # 2
    count += getLayerDepth(model.layer2)
    modules.append(model.layer2)
    names.append('layer2')
    depths.append(count)      
    # 3
    count += getLayerDepth(model.layer3)
    modules.append(model.layer3)
    names.append('layer3')
    depths.append(count)     
    # 4 
    count += getLayerDepth(model.layer4)
    modules.append(model.layer4)
    names.append('layer4')
    depths.append(count)      
    # average pooling
    count += 1
    modules.append(model.avgpool)
    names.append('avgpool')
    depths.append(count)     
    # output
    count += 1
    modules.append(model.fc)
    names.append('fc')
    depths.append(count)                      
    depths = np.array(depths)    
    return modules, names, depths

N.of classes : 0


In [5]:
#model training
model = vgg16(pretrained=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# this switch to evaluation mode your network: in this way dropout and batchnorm 
# no more active and you can use the network as a 'passive' feedforward device; 
# forgetting this produces catastrophically wrong results (I know because I did it)
model.eval()
print('Training mode : {}'.format(model.training))

modules, names, depths = getDepths(model)
print('List of layers from which to extract representations: {}'.format(names))


mean_imgs = [0.485, 0.456, 0.406]
std_imgs = [0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# Data transformations (same as suggested by Soumith Chintala's script)
data_transforms =  transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

Training mode : False
List of layers from which to extract representations: ['input', 'MaxPool2d', 'MaxPool2d', 'MaxPool2d', 'MaxPool2d', 'MaxPool2d', 'Linear', 'Linear', 'Linear']


In [ ]:
model


## Extraction of representations
This will extract for each class the representations of $\sim 500$ images from all checkpoints, including input and output.

It will save these representations as matrices of shape (n.images,embedding dimension).

A typical filename will be n02086240_5.npy which means that this file contains the representations of class n02086240 extracted at the sixt (5+1) checkpoint layer, which is the max pooling after the last convolutional layer, as you can easily check by printing the list of names.

In [7]:

for i,name in enumerate(names):
    print(i,name)
    

0 input
1 MaxPool2d
2 MaxPool2d
3 MaxPool2d
4 MaxPool2d
5 MaxPool2d
6 Linear
7 Linear
8 Linear


In [6]:

for i,name in enumerate(names):
    print(i,name)
    
n_layers = len(modules)
embdims=[] # store the embedding dimension of the checkpoint layers (n.of units)


for i,tag in enumerate(category_tags):
    
    print('Processing class: {}'.format(tag))
    data_folder = join(ROOT, 'data', 'imagenet_training_single_objs', tag)
    image_dataset = datasets.ImageFolder(join(data_folder), data_transforms)           
    dataloader = torch.utils.data.DataLoader(image_dataset, 
                                             batch_size=bs, 
                                             shuffle=True, 
                                             num_workers=1)  

    for l,module in enumerate(modules):    
        for k, data in enumerate(dataloader, 0):
            if k*bs > nsamples:
                break
            else:  
                inputs, _ = data                          
                if module == 'input':                
                    hout = inputs                      
                else:            
                    hout = []
                    def hook(module, input, output):
                        hout.append(output)                
                    handle = module.register_forward_hook(hook)                            
                    out = model(inputs.to(device))
                    del out   
                    
                    hout = hout[0] 
                            
                    handle.remove()

                if k == 0:
                    Out = hout.view(inputs.shape[0], -1).cpu().data    
                else :               
                    Out = torch.cat((Out, hout.view(inputs.shape[0], -1).cpu().data),0) 
                hout = hout.detach().cpu()
                del hout

        Out = Out.detach().cpu()  
        embdims.append(Out.shape[1])
        print(tag+"_"+str(l))
        np.save(join(results_folder, tag + '_' + str(l) + '.npy' ), Out)

0 input
1 MaxPool2d
2 MaxPool2d
3 MaxPool2d
4 MaxPool2d
5 MaxPool2d
6 Linear
7 Linear
8 Linear
Processing class: mix
mix_0
mix_1
mix_2
mix_3
mix_4
mix_5
mix_6
mix_7
mix_8


In [ ]:
model.eval()
print(model.training)
use_gpu = torch.cuda.is_available()

extract = True
if extract:
    verbose = False
    for i, data in tqdm(enumerate(dataloader, 0)): 
        inputs, _ = data                                              
        inputs = Variable(inputs.cuda())           
        out0,out1,out2,out3,out4,out5,out6,out7,out8 = model.extract_all(inputs, verbose)  
        if i == 0:
            Out0 = out0.view(inputs.shape[0], -1).cpu().data 
            Out1 = out1.view(inputs.shape[0], -1).cpu().data 
            Out2 = out2.view(inputs.shape[0], -1).cpu().data 
            Out3 = out3.view(inputs.shape[0], -1).cpu().data                 
            Out4 = out4.view(inputs.shape[0], -1).cpu().data  
            Out5 = out5.view(inputs.shape[0], -1).cpu().data 
            Out6 = out6.view(inputs.shape[0], -1).cpu().data 
            Out7 = out7.view(inputs.shape[0], -1).cpu().data                 
            Out8 = out8.view(inputs.shape[0], -1).cpu().data          
        else :    
            Out0 = torch.cat((Out0, out0.view(inputs.shape[0], -1).cpu().data),0)
            Out1 = torch.cat((Out1, out1.view(inputs.shape[0], -1).cpu().data),0) 
            Out2 = torch.cat((Out2, out2.view(inputs.shape[0], -1).cpu().data),0) 
            Out3 = torch.cat((Out3, out3.view(inputs.shape[0], -1).cpu().data),0)                 
            Out4 = torch.cat((Out4, out4.view(inputs.shape[0], -1).cpu().data),0) 
            Out5 = torch.cat((Out5, out5.view(inputs.shape[0], -1).cpu().data),0) 
            Out6 = torch.cat((Out6, out6.view(inputs.shape[0], -1).cpu().data),0) 
            Out7 = torch.cat((Out7, out7.view(inputs.shape[0], -1).cpu().data),0)                 
            Out8 = torch.cat((Out8, out8.view(inputs.shape[0], -1).cpu().data),0) 

    # save representations

    print(Out0.shape)
    print(Out1.shape)
    print(Out2.shape)
    print(Out3.shape)
    print(Out4.shape)
    print(Out5.shape)
    print(Out6.shape)
    print(Out7.shape)
    print(Out8.shape)


    torch.save(Out0, join(results_folder, 'Out0') )
    torch.save(Out1, join(results_folder, 'Out1') )
    torch.save(Out2, join(results_folder, 'Out2') )
    torch.save(Out3, join(results_folder, 'Out3') )
    torch.save(Out4, join(results_folder, 'Out4') )
    torch.save(Out5, join(results_folder, 'Out5') )
    torch.save(Out6, join(results_folder, 'Out6') )
    torch.save(Out7, join(results_folder, 'Out7') )
    torch.save(Out8, join(results_folder, 'Out8') )
    
    del Out0,Out1,Out2,Out3,Out4,Out5,Out6,Out7,Out8
    